# Snowflake as Data Source for training an ML Model with Amazon Sagemaker
**_Use of Snowflake Data Table as Data Source for training a Sagemaker Model without having Snowflake Data to stage on S3_**

This notebook works well with the `conda_python3` kernel on a SageMaker Notebook `ml.t3.xlarge` instance.

---
---

## Contents

1. [Objective](#Objective)
1. [Background](#Background-(Problem-Description-and-Approach))
1. [Create a Training Script](#Create-a-training-script)
1. [Define `Model` Hyperparameters](#Define-Model-Hyperparameters)
1. [Launch a training job with Python SDK](#Launch-a-training-job-with-Python-SDK)
1. [Conclusion](#Conclusion)

---

## Objective

This notebook illustrates how to retrieve data stored in a [Snowflake](https://www.snowflake.com/) table and use it for training an ML model using Amazon SageMaker _without having to first store the data in S3_. 

This example uses the [California Housing dataset (provided by Scikit-Learn)](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) and trains a XGBoost model to predict house prices. A detailed description about the dataset can be found [here](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html).

To understand the code, you might also find it useful to refer to:

- *The guide on [Use XGBoost with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/using_xgboost.html#)*
- *Docker Registry Paths [Docker Registry Paths and Example Code](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html)*
- *The [SageMaker reference for Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#client) (The general AWS SDK for Python, including low-level bindings for SageMaker as well as many other AWS services)*
---

## Background (Problem Description and Approach)

- **Problem statement**: SageMaker requires the training data to be present either in [S3 or in EFS or in FSX for Lustre](https://aws.amazon.com/blogs/machine-learning/choose-the-best-data-source-for-your-amazon-sagemaker-training-job/). In order to train a model using data stored outside of the three supported storage services, the data first needs to be ingested into one of these services (typically S3). This requires building a data pipeline (using tools such as [Amazon SageMaker Data Wrangler](https://aws.amazon.com/sagemaker/data-wrangler/)) to move data into S3. However, this may create a data management challenge in some situations (data lifecycle management, access control etc.) and it may be desirable to have the data accessible to SageMaker _without_ the intermediate storage of data into S3. This notebook illustrates a way to do this using Snowflake as a 3rd party data source.

- **Our approach**: Launch a SageMaker Training Job using the SageMaker SDK with a custom training script and have the training script download the data from Snowflake directly into the instance created for running the training job thus avoiding the temporary storage of data in S3. Use the [Snowflake Connector for Python](https://docs.snowflake.com/en/user-guide/python-connector.html) to connect and download the data from the Snowflake. Once the data is downloaded in the training instance then the training script proceeds to train an ML model using the scikit-learn SDK. **Note that it is assumed that the data is already available in Snowflake, see [`snowflake instructions`](./snowflake-instructions.md) for instructions on creating a database in Snowflake and ingesting the California Housing dataset as a table.** We create and use a custom container for SageMaker XGBoost so that we can package a script to download credentials from Snowflake into this container from the AWS Secrets Mananger. This approach fres up the data scientists from being aware about how these credentials are stored and how to retrieve. 

- **Our tools**: [Amazon SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/), [Snowflake Connector for Python](https://docs.snowflake.com/en/user-guide/python-connector.html) and [SageMaker XGBoost Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/using_xgboost.html#).


---

## Overall Workflow

The overall workflow for this notebook is shown in the diagram below.

![sagemaker snowflake](img/sm-snowflake.png)

Steps 1 and 2 are executed outside of this notebook. 

1. See [`snowflake instructions`](./snowflake-instructions.md) for instructions on creating a database in Snowflake and ingesting the California Housing dataset as a table.
1. See [`secrets manager instructions`](./secretsmanager-instructions.md) for instructions on storing Snowflake credentials.

The following sections in this notebook describe steps 3 and 4.

---

## Create a custom container image

To create a custom container image we use the SageMaker XGBoost container image as the base and then add the following:

1. [Snowflake Connector for Python](https://docs.snowflake.com/en/user-guide/python-connector.html) to the image.
1. A script to download credentials from the AWS Secrets Mananger, these credentials are used to connect to Snowflake.

In [20]:
!pip install boto3==1.26.44 sagemaker==2.127.0

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [21]:
"""
Create directories for storing scripts and Dockerfile
"""
!mkdir src
!mkdir scripts
!mkdir container

mkdir: cannot create directory ‘src’: File exists
mkdir: cannot create directory ‘scripts’: File exists
mkdir: cannot create directory ‘container’: File exists


### Script for downloading credentials

This script is written to a file locally and then packaged inside the container.

In [22]:
%%writefile container/snowflake_credentials.py

"""
Retrieve Snowflake password for given username from AWS SecretsManager
"""
import json
import boto3

def get_credentials(secret_id: str, region_name: str) -> str:
    
    client = boto3.client('secretsmanager', region_name=region_name)
    response = client.get_secret_value(SecretId=secret_id)
    secrets_value = json.loads(response['SecretString'])    
    
    return secrets_value


Overwriting container/snowflake_credentials.py


### Dockerfile

This is the Dockerfile used for creating the custom container used for training. Note the use of `246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.5-1` as the base image for this container. We are packaging `snowflake-connector-python==2.8.3` and `snowflake_credentials.py` script we created above into this container.

In [23]:
%%writefile container/Dockerfile

# Build an image that can be used for training in Amazon SageMaker, we use
# the SageMaker XGBoost as the base image as it contains support for distributed
# training.
FROM 246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.5-1

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip

# Here we get snowflake-connector python package.
# pip leaves the install caches populated which uses a 
# significant amount of space. These optimizations save a fair 
# amount of space in the image, which reduces start up time.
#RUN pip --no-cache-dir install snowflake-connector-python==2.8.3
RUN pip --no-cache-dir install snowflake-connector-python==2.9.0

# Include python script for retrieving Snowflake credentials 
# from AWS SecretsManager
ADD snowflake_credentials.py /

# Set some environment variables. PYTHONUNBUFFERED keeps Python from buffering our standard
# output stream, which means that logs can be delivered to the user quickly. PYTHONDONTWRITEBYTECODE
# keeps Python from writing the .pyc files which are unnecessary in this case. We also update
# PATH so that the train and serve programs are found when the container is invoked.
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE


Overwriting container/Dockerfile


### Script to push the container image to Amazon ECR

We are now going to build the container image and push to our container registry i.e. Amazon ECR. This image will be used for downloading data from Snowflake, doing data preparation and finally training the ML model.

In [24]:
%%writefile scripts/build_and_push.sh

#!/usr/bin/env bash
# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.
# The argument to this script are the path to the Dockerfile, the image name and tag and the aws-region
# in which the container is to be created. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.

# override the built-in echo so that we can have a nice timestamped trace
echo () {
    builtin echo "$(date +'[%m-%d %H:%M:%S]'):" "$@"
}

if [ "$#" -eq 4 ]; then
    dlc_account_id=$(aws sts get-caller-identity | jq .Account)
    path_to_dockerfile=$1
    image=$2
    tag=$3
    region=$4
    
else
    echo "missing mandatory command line arguments, see usage..."
    echo "usage: $0 </path/to/Dockerfile> $1 <image-repo> $2 <image-tag> $3 <aws-region>"
    exit 1
fi

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"
echo the full image name would be ${fullname}

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    echo "creating ECR repository : ${fullname} "
    aws ecr create-repository --region ${region} --repository-name "${image}" > /dev/null
else
    echo "${image} repo already exists in ECR"
fi

# move to path of dockerfile
cd ${path_to_dockerfile}

# get credentials to login to ECR and, build and tag the image
# note the use of DOCKER_BUILDKIT=1, this is needed for some mount instructions in the Dockerfile
echo "going to start a docker build, image=${image}, using Dockerfile=${path_to_dockerfile}"
aws ecr get-login-password --region ${region} \
| docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com
DOCKER_BUILDKIT=1 docker build . -t ${image}  --build-arg dlc_account_id=${dlc_account_id} --build-arg region=${region}
docker tag ${image} ${fullname}
echo ${image} created

# push the image to ECR
cmd="aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com"
echo going to run \"${cmd}\" to login to ECR
${cmd}

cmd="docker push ${fullname}"
echo going to run \"${cmd}\" to push image to ecr
${cmd}
if [ $? -eq 0 ]; then
    echo "Amazon ECR URI: ${fullname}"
else
    echo "Error: Image ${fullname} build and push failed"
    exit 1
fi

echo "all done"

Overwriting scripts/build_and_push.sh


In [25]:
# Get credentials and login to ECR to pull XGBoost image from public AWS repository  
import boto3
session = boto3.session.Session()
region = session.region_name
print(f"region={region}")
sagemaker_xgboost_repo="246618743249.dkr.ecr.us-west-2.amazonaws.com"
! aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin $sagemaker_xgboost_repo

region=us-east-1
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [26]:
# Run script to build docker custom containe image and push it to ECR 
# Set region and sagemaker URI variables 
import boto3
import sagemaker
from sagemaker.session import Session

!bash scripts/build_and_push.sh $(pwd)/container xgboost-ddp-training-custom latest $region

[01-24 06:36:53]: the full image name would be 328296961357.dkr.ecr.us-east-1.amazonaws.com/xgboost-ddp-training-custom:latest
[01-24 06:36:54]: xgboost-ddp-training-custom repo already exists in ECR
[01-24 06:36:54]: going to start a docker build, image=xgboost-ddp-training-custom, using Dockerfile=/home/ec2-user/SageMaker/snowflake-sermolin-review-2/container
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.50kB                                     0.0s
 => [internal] load .dockerignore                                       

---

## Train an XGBoost Regressor using SageMaker Training Jobs

Now we are ready to train our ML model using SageMaker Training Jobs. We do this in the following steps:

1. Create separate Python scripts for connecting to Snowflake, querying (downloading) the data, preparing the data for ML and finally a training scripts which ties everything together.

1. Provide the training script to the SageMaker SDK [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) along with the source directory so that all the scripts we create can be provided to the training container when the training job is run using the [Estimator.fit](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit) method.

**NOTE: the data from Snowflake is downloaded directly into the training container instance and at no point is it stored in S3.**

### Create a Snowflake connection Script

Use the [Snowflake Connector for Python](https://docs.snowflake.com/en/user-guide/python-connector.html) to connect and download the data from the Snowflake.

In [27]:
%%writefile src/connection.py

"""
Establish connection with Snowflake table - HOUSING
"""

import snowflake.connector
import snowflake_credentials
from snowflake_credentials import get_credentials

def connect(secret_id: str, account: str, warehouse: str, database: str, schema: str, protocol: str, region: str) -> snowflake.connector.SnowflakeConnection:
    
    secret_value = get_credentials(secret_id, region)
    sf_user = secret_value['username']
    sf_password = secret_value['password']
    sf_account = account
    sf_warehouse = warehouse
    sf_database = database
    sf_schema = schema
    sf_protocol = protocol
    
    print(f"sf_user={sf_user}, sf_password=****, sf_account={sf_account}, sf_warehouse={sf_warehouse}, "
          f"sf_database={sf_database}, sf_schema={sf_schema}, sf_protocol={sf_protocol}")    
    
    # Read to connect to snowflake
    ctx = snowflake.connector.connect(user=sf_user,
                                      password=sf_password,
                                      account=sf_account,
                                      warehouse=sf_warehouse,
                                      database=sf_database,
                                      schema=sf_schema,
                                      protocol=sf_protocol)
    
    # Once the connection is established we read the dataset (table)
    # into a dataframe
    cs=ctx.cursor()
    print("\nSnowflake connection established...")
    
    return ctx

Overwriting src/connection.py


### Create a Snowflake Querying Script

To query data records from Snowflake database table, and store it in a dataframe.

**For distributed data parallel training we download a random subset of data into each training instance. Each training instance downloads an equal amount of data which is simply `total number of rows/ total number of training hosts`**


In [28]:
%%writefile src/query_snowflake.py

"""
Read the HOUSING table (this is the california housing dataset used by this example)
"""
import pandas as pd
import snowflake.connector

def data_pull(ctx: snowflake.connector.SnowflakeConnection, table: str, hosts: int) -> pd.DataFrame:
    
    # Query Snowflake HOUSING table for number of table records
    sql_cnt = f"select count(*) from {table};"
    df_cnt = pd.read_sql(sql_cnt, ctx)

    # Retrieve the total number of table records from dataframe
    for index, row in df_cnt.iterrows():
        num_of_records = row.astype(int)
        list_num_of_rec = num_of_records.tolist()
    tot_num_records = list_num_of_rec[0]

    record_percent = str(round(100/hosts))
    print(f"going to download a random {record_percent}% sample of the data")
    # Query Snowflake HOUSING table
    sql = f"select * from {table} sample ({record_percent});"
    print(f"sql={sql}")
    
    # Get the dataset into Pandas
    df = pd.read_sql(sql, ctx)
    print(f"read data into a dataframe of shape {df.shape}")
    # Prepare the data for ML
    df.dropna(inplace=True)
    
    print(f"Number of rows after dropping NA: {df.shape[0]}")
    """
    df['AVG_ROOMS'] = (df['TOTAL_ROOMS']/df['HOUSEHOLDS']).round(6)
    df['AVG_BEDROOMS'] = (df['TOTAL_BEDROOMS']/df['HOUSEHOLDS']).round(6)
    df['AVE_OCCUP'] = (df['POPULATION']/df['HOUSEHOLDS']).round(6)
    df.drop(['OCEAN_PROXIMITY'], axis = 1, inplace=True)
    df.drop(['TOTAL_ROOMS'], axis = 1, inplace=True)
    df.drop(['TOTAL_BEDROOMS'], axis = 1, inplace=True)
    df.drop(['HOUSEHOLDS'], axis = 1, inplace=True)
    """
    print(f"final shape of dataframe to be used for training {df.shape}")
    return df

Overwriting src/query_snowflake.py


### Create a Data Preparation Script

The input dataframe is split into training and test datasets using [SKlearn's train_test_split function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) The split datasets are then converted to [XGB DMatrices](https://xgboost.readthedocs.io/en/stable/python/python_api.html).

In [29]:
%%writefile src/data_preparation.py

"""
Preparation of training and test datasets for XGBoost Model
"""
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

def prepare_data(df: pd.DataFrame) -> tuple:
    
    # preparation of training and test datasets
    X = df.drop(['MEDHOUSEVAL'], axis=1)  
    y = df['MEDHOUSEVAL']
    
    # Train-test split
    X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.25, random_state=42)
    
    print("building training and testing datasets")
    features = X.select_dtypes('number').columns.tolist()
    print(f"features={features}")
    X_train = X_train[features]
    X_test = X_test[features]

    # -- MODEL TRAINING --    
    print("going to train the model")
    
    # Converting input datasets to XGB DMatrices for XGBoost Model Training
    dtrain = xgb.DMatrix(X_train.values, y_train.values)
    dval = xgb.DMatrix(X_test.values, y_test.values)
    watchlist = [(dtrain, 'train'), (dval, 'validation')] if dval is not None else [(dtrain, 'train')]
    
    return dtrain, dval, watchlist

Overwriting src/data_preparation.py


### Create a Training Script

The [SageMaker Scikit-Learn Framework Container](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-docker-containers-scikit-learn-spark.html) provides the basic runtime, and we provide a custom [`training script`](./src/train.py) that contains the actual ML training code. The folder containing the training script can optionally also contain a [`requirements.txt`](./src/requirements.txt) file to specify any additional dependencies that need to be installed in the training instance.

You can find detailed guidance in the documentation on [Preparing a Scikit-Learn training script](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#prepare-a-scikit-learn-training-script) (for training).


In [30]:
%%writefile src/train.py

"""
Train a Sagemaker XGBoost Model on the california housing dataset
"""

# Import dependencies
import os
import json
import logging
import argparse
import pickle as pkl
import xgboost as xgb
from connection import connect
from query_snowflake import data_pull
from data_preparation import prepare_data
from sagemaker_containers import entry_point
from sagemaker_xgboost_container import distributed
from sagemaker_xgboost_container.data_utils import get_dmatrix

def _xgb_train(params: dict, dtrain: xgb.DMatrix, evals: list, num_boost_round: int, model_dir: str, is_master: bool) -> None:
    """Run xgb train on arguments given with rabit initialized.

    This is our rabit execution function.

    :param args_dict: Argument dictionary used to run xgb.train().
    :param is_master: True if current node is master host in distributed training,
                        or is running single node training job.
                        Note that rabit_run includes this argument.
    """
    booster = xgb.train(params=params,
                        dtrain=dtrain,
                        evals=evals,
                        num_boost_round=num_boost_round)

    if is_master:
        model_location = model_dir + '/xgboost-model'
        pkl.dump(booster, open(model_location, 'wb'))
        logging.info("Stored trained model at {}".format(model_location))

    
if __name__ == "__main__":
    
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here.
    parser.add_argument('--max_depth', type=int,)
    parser.add_argument('--eta', type=float)
    parser.add_argument('--gamma', type=int)
    parser.add_argument('--min_child_weight', type=int)
    parser.add_argument('--subsample', type=float)
    parser.add_argument('--verbosity', type=int)
    parser.add_argument('--objective', type=str)
    parser.add_argument('--num_round', type=int)
    parser.add_argument('--tree_method', type=str, default="auto")
    parser.add_argument('--predictor', type=str, default="auto")

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output_data_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--sm_hosts', type=str, default=os.environ.get('SM_HOSTS'))
    parser.add_argument('--sm_current_host', type=str, default=os.environ.get('SM_CURRENT_HOST'))

    args, _ = parser.parse_known_args()

    # Get SageMaker host information from runtime environment variables
    sm_hosts = json.loads(args.sm_hosts)
    sm_current_host = args.sm_current_host   
    
    # snowflake related params are read from environment variables
    secret_id = os.environ["SECRET_ID"]
    account = os.environ["SF_ACCOUNT"]
    warehouse = os.environ["SF_WAREHOUSE"]
    database = os.environ["SF_DATABASE"].upper()
    schema = os.environ["SF_SCHEMA"].upper()
    table = os.environ['SF_TABLE'].upper()
    region = os.environ["AWS_REGION"]
    
    protocol = "https"
    
    # Connect to Snowflake database table 
    ctx = connect(secret_id, account, warehouse, database, schema, protocol, region)
    
    # Query data from Snowflake database table
    # IMPORTANT: DATA FROM SNOWFLAKE GOES DIRECTLY INTO PANDA DF. THE DATA DOES __NOT__ GET STAGED IN AND S3 BUCKET. 
    df = data_pull(ctx, table, len(sm_hosts))
    
    # Preparation of training and test datasets
    dtrain, dval, watchlist = prepare_data(df)
    
    # Define training hyperparameters
    train_hp = {
        'max_depth': args.max_depth,
        'eta': args.eta,
        'gamma': args.gamma,
        'min_child_weight': args.min_child_weight,
        'subsample': args.subsample,
        'verbosity': args.verbosity,
        'objective': args.objective,
        'tree_method': args.tree_method,
        'predictor': args.predictor,
    }
    
    xgb_train_args = dict(
        params=train_hp,
        dtrain=dtrain,
        evals=watchlist,
        num_boost_round=args.num_round,
        model_dir=args.model_dir)

    if len(sm_hosts) > 1:
        # Wait until all hosts are able to find each other
        entry_point._wait_hostname_resolution()

        # Execute training function after initializing rabit.
        distributed.rabit_run(
            exec_fun=_xgb_train,
            args=xgb_train_args,
            include_in_training=(dtrain is not None),
            hosts=sm_hosts,
            current_host=sm_current_host,
            update_rabit_args=True
        )
    else:
        # If single node training, call training method directly.
        if dtrain:
            xgb_train_args['is_master'] = True
            _xgb_train(**xgb_train_args)
        else:
            raise ValueError("Training channel must have data to train model.")


def model_fn(model_dir: str):
    """Deserialize and return fitted model.

    Note that this should have the same name as the serialized model in the _xgb_train method
    """
    model_file = 'xgboost-model'
    booster = pkl.load(open(os.path.join(model_dir, model_file), 'rb'))
    return booster

Overwriting src/train.py


#### Provide the Snowflake username and connection details as environment variables to the Training container

Retrieve the Snowflake credentials from AWS Secrets Mananger. This is done by the training code and we just need to provide the secrets identifier as an environment variable.

We also need to retrieve your account identifier for Snowflake which we already obtained when we run the previous notebook (snowflake-load-dataset.ipynb).

In [31]:
#secret_id = "snowflake_credentials"
#account = "your-snowflake-account-id"

%store -r sf_account_id 
%store -r snowflake_secret 

secret_id=snowflake_secret
account=sf_account_id

In [32]:
print (sf_account_id, snowflake_secret)
print (secret_id, account)

uub56114.us-east-1 snowflake-credentials
snowflake-credentials uub56114.us-east-1


In [33]:
## TODO - replace with -r magic, make naming consistent. 
# do not change!!!
# the values of these variables match what we put in the snowflake-load-dataset.ipynb file
warehouse = "amazon_sagemake_w_snowflake_as_datasource"
database = "housing"
schema = "housing_schema"
table = "california_housing"

env = {"SECRET_ID": secret_id, 
       "SF_ACCOUNT": account,
       "SF_WAREHOUSE": warehouse,
       "SF_DATABASE": database,
       "SF_SCHEMA": schema,
       "SF_TABLE": table,
       "AWS_REGION": region}

### Define Model Hyperparameters

In [34]:
hyperparams = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "reg:squarederror",
    "num_round": "50",
    "verbosity": "2",
}

### Distributed data parallel training

For distributed data parallel training we set the `instance_count > 1` and provide an qual amount of random subset of the data to each training instance.

In [35]:
instance_type = "ml.m5.2xlarge"
instance_count = 2

### Launch a training job

With the data uploaded and script prepared, we are ready to configure SageMaker training jobs.

In [36]:
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

role = get_execution_role()
sm_session = sagemaker.Session()
bucket = None #optionally specify you bucket here, eg: 'mybucket-us-east-1'; Otherwise, SageMaker will use default acct bucket
if bucket is None and sm_session is not None:
    bucket = sm_session.default_bucket()
print(f"bucket={bucket}, role={role}")
prefix = "sagemaker/sagemaker-snowflake-example"
output_path = "s3://{}/{}/{}/output".format(bucket, prefix, "housing-dist-xgb")
custom_img_name = "xgboost-ddp-training-custom"
custom_img_tag = "latest"
account_id = boto3.client('sts').get_caller_identity().get('Account')

# collect default subnet IDs to deploy Sagemaker training job into
ec2_session = boto3.Session(region_name=region)
ec2_resource = ec2_session.resource("ec2")
subnet_ids = []
for vpc in ec2_resource.vpcs.all():
    # here you can choose which subnet based on the id
    if vpc.is_default == True:
        for subnet in vpc.subnets.all():
            if subnet.default_for_az == True:
                subnet_ids.append(subnet.id)

# Retrieve XGBoost custom container from ECR registry path 
# custom_img_uri = account_id+".dkr.ecr."+region+".amazonaws.com/"+custom_img_name+":"+custom_img_tag
custom_img_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{custom_img_name}:{custom_img_tag}"
print(f"\nusing custom image: {custom_img_uri}")

# Create Sagemaker Estimator
xgb_script_mode_estimator = sagemaker.estimator.Estimator(
    image_uri = custom_img_uri,
    role=role,
    instance_count=instance_count,
    instance_type=instance_type,
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sm_session,
    entry_point="train.py",
    source_dir="./src",
    hyperparameters=hyperparams,
    environment=env,
    subnets = subnet_ids,
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


bucket=sagemaker-us-east-1-328296961357, role=arn:aws:iam::328296961357:role/SageMakerSnowFlakeExampleIAMRole-03

using custom image: 328296961357.dkr.ecr.us-east-1.amazonaws.com/xgboost-ddp-training-custom:latest


In [37]:
# Estimator fitting
xgb_script_mode_estimator.fit()

INFO:sagemaker:Creating training-job with name: xgboost-ddp-training-custom-2023-01-24-06-36-58-181


2023-01-24 06:36:58 Starting - Starting the training job...
2023-01-24 06:37:15 Starting - Preparing the instances for training......
2023-01-24 06:38:22 Downloading - Downloading input data
2023-01-24 06:38:22 Training - Downloading the training image...
2023-01-24 06:38:53 Training - Training image download completed. Training in progress....[2023-01-24 06:39:13.954 ip-10-2-231-100.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-01-24:06:39:13:INFO] Imported framework sagemaker_xgboost_container.training
[2023-01-24:06:39:13:INFO] No GPUs detected (normal if no gpus installed)
[2023-01-24:06:39:13:INFO] Invoking user training script.
[2023-01-24:06:39:14:INFO] Module train does not provide a setup.py. 
Generating setup.py
[2023-01-24:06:39:14:INFO] Generating setup.cfg
[2023-01-24:06:39:14:INFO] Generating MANIFEST.in
[2023-01-24:06:39:14:INFO] Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  

[06:39:25] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 10 pruned nodes, max_depth=5
[06:39:25] INFO: ../src/gbm/gbtree.cc:138: Tree method is automatically selected to be 'approx' for distributed training.
[06:39:25] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 10 pruned nodes, max_depth=5
[06:39:25] INFO: ../src/gbm/gbtree.cc:138: Tree method is automatically selected to be 'approx' for distributed training.
[06:39:25] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 22 pruned nodes, max_depth=5
[06:39:25] INFO: ../src/gbm/gbtree.cc:138: Tree method is automatically selected to be 'approx' for distributed training.
[06:39:25] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 14 pruned nodes, max_depth=5
[06:39:25] INFO: ../src/gbm/gbtree.cc:138: Tree method is automatically selected to be 'approx' for distributed training.
[06:39:26] INFO: ../src/tree/updater_prune.cc:101: tree 

[06:39:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 12 extra nodes, 34 pruned nodes, max_depth=5
[06:39:27] INFO: ../src/gbm/gbtree.cc:138: Tree method is automatically selected to be 'approx' for distributed training.
[06:39:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 34 pruned nodes, max_depth=5
[06:39:27] INFO: ../src/gbm/gbtree.cc:138: Tree method is automatically selected to be 'approx' for distributed training.
[06:39:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 36 pruned nodes, max_depth=5
[06:39:27] INFO: ../src/gbm/gbtree.cc:138: Tree method is automatically selected to be 'approx' for distributed training.
[06:39:28] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 10 extra nodes, 36 pruned nodes, max_depth=5

2023-01-24 06:39:33 Uploading - Uploading generated training model
2023-01-24 06:40:11 Completed - Training job completed
Training seconds: 246
Billable seconds: 246


In [38]:
print(f"the trained model is available in S3 -> {xgb_script_mode_estimator.model_data}")

the trained model is available in S3 -> s3://sagemaker-us-east-1-328296961357/sagemaker/sagemaker-snowflake-example/output/xgboost-ddp-training-custom-2023-01-24-06-36-58-181/output/model.tar.gz




Remember that the training job that we ran is very "light", due to the very small dataset. As such, running locally on the notebook instance results in a faster execution time, compared to SageMaker. SageMaker takes longer time to run the job because it has to provision the training infrastructure. Since this example training job not very resource-intensive, the infrastructure provisioning process adds more overhead, compared to the training job itself.

In a real situation, where datasets are large, running on SageMaker can considerably speed up the execution process - and help us optimize costs, by keeping this interactive notebook environment modest and spinning up more powerful training job resources on-demand.

---

## Cleaning up

To avoid incurring future charges, delete the resources. You can do this by deleting the cloud formation template used to create the IAM role and the Amazon SageMaker Notebook.
![Cleaning Up](img/cfn-delete.png)

You will have to delete the Snowflake resources manually from the Snowflake console.

---

## Conclusion

In this notebook we saw how to download data stored in Snowflake table to Sagemaker Training job instance and train a XGBoost model using a custom training container. **This approach allows us to directly integrate Snowflake as a data source with Sagemaker notebook without having the data staged on S3.**